In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


# モデル読み込み

In [ ]:
from model.Temperature.Laser import LaserProfile

# テスト

In [ ]:
# 初期化
laser = LaserProfile()

In [ ]:
dir(laser)

# パラメータ設定

In [ ]:
# Linearの場合
laser.start_power = 10
laser.goal_power = 28

laser.width = 18_000

In [ ]:
laser.get_plot_values()

In [ ]:
laser.show_profile_plot()

# develop

In [ ]:
laser.step_num

In [ ]:
laser.energy_step

In [ ]:
laser.time_series/1000

In [ ]:
laser.time_series.shape

In [ ]:
x = np.arange(laser.time_series.shape[0])
plt.scatter(x, laser.time_series/1000)
plt.xlim(0, 10)

In [ ]:
# 時間配列

# start
time_arr = np.zeros(1)

# 加熱はじめ
time_arr = np.r_[time_arr, laser.delay + laser.time_adjust]
time_arr = np.r_[time_arr, laser.delay + laser.time_adjust]

# step加熱
# FIXME
#   time_adjustは一回にする
#   time_series[i+1]がわかりづらい。計算する
for i in range(int(laser.step_num)):
    time_arr = np.r_[time_arr, laser.time_series[i+1]+ laser.time_adjust]
    time_arr = np.r_[time_arr, laser.time_series[i+1]+ laser.time_adjust]

# 加熱終わり
time_arr = np.r_[time_arr, time_arr[-1]] # クエンチ
end_time = laser.delay + laser.width + laser.residue
time_arr = np.r_[time_arr, end_time]

time_arr /= 1000
time_arr

In [ ]:
# エネルギー配列

# start
energy_arr = np.zeros(1)

# 加熱はじめ
energy_arr = np.r_[energy_arr, np.array([0])]
energy_arr = np.r_[energy_arr, laser.start_power]

# step加熱
energy = laser.start_power
for i in range(int(laser.step_num)):
    energy_arr = np.r_[energy_arr, energy]
    energy += laser.energy_step
    energy_arr = np.r_[energy_arr, energy]

# 加熱終わり
energy_arr = np.r_[energy_arr, np.zeros(1)]
energy_arr = np.r_[energy_arr, np.zeros(1)]

energy_arr

In [ ]:
plt.figure(figsize=(6,6), dpi=1000)
plt.plot(time_arr, energy_arr, lw=0.5)  